In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import pprint
import re
import numpy as np
import math

In [2]:
df = pd.read_pickle("./data/article_contents_20211010.pkl")

In [3]:
df.head(30)  

,title,mdName,content,url
0,"대구보건대 안경광학과, 현장직무수행능력 평가 실시",헤럴드경제,지난 6일 열린 대구보건대 안경광학과 현장직무수행능력 평가 모습.[대구보건대 제공]...,https://news.naver.com/main/read.naver?mode=LS...
1,ITALY SOCCER UEFA NATIONS LEAGUE,EPA연합뉴스,Italy vs BelgiumDomenico Berardi (bottom) of I...,https://news.naver.com/main/read.naver?mode=LS...
2,EPA OUTLOOK,EPA연합뉴스,EPA OUTLOOKWe expect to move pictures on the f...,https://news.naver.com/main/read.naver?mode=LS...
3,ITALY SOCCER UEFA NATIONS LEAGUE,EPA연합뉴스,Italy vs BelgiumKevin De Bruyne (C) of Belgium...,https://news.naver.com/main/read.naver?mode=LS...
4,"경북도, 한글비전 선포식 개최",헤럴드경제,경북도 한글비전 선포식 개최 모습.[경북도 제공] [헤럴드경제(안동)=김병진 기자]...,https://news.naver.com/main/read.naver?mode=LS...
5,"이낙연 측, ""무효표 이의제기""...경선 후유증 현실화되나",YTN,동영상 뉴스\n\n\n\n\n\t\n\t[앵커]민주당 대선 후보로 이재명 경기지사가...,https://news.naver.com/main/read.naver?mode=LS...
6,Turkey F1 GP Auto Racing,AP연합뉴스,Mercedes driver Valtteri Bottas of Finland ste...,https://news.naver.com/main/read.naver?mode=LS...
7,예상 뒤엎은 아슬아슬 과반...결국 대장동 의혹 영향,YTN,동영상 뉴스\n\n\n\n\n\t\n\t[앵커]이재명 후보는 최종 누적 득표율 50...,https://news.naver.com/main/read.naver?mode=LS...
8,"롯데백화점 대구점, 가을맞이 라이프 스타일 홈페어 개최",헤럴드경제,[롯데백화점 제공][헤럴드경제(대구)=김병진 기자]사회적 거리두기 강화 방침이 계속...,https://news.naver.com/main/read.naver?mode=LS...
9,Italy Belgium Nations League Soccer,AP연합뉴스,"Belgium's Charles De Ketelaere, left, celebrat...",https://news.naver.com/main/read.naver?mode=LS...


In [ ]:
vocab_dict = {}
idx = 0
for i, row in df.iterrows():
    title = row['title']
    
    terms = title.split()
    for term in terms:
        if term not in vocab_dict:
            vocab_dict[term] = idx
            idx+=1
vocab_dict

In [5]:
len(vocab_dict)

20274

In [6]:
def title2idx(title, vocab_dict):
    result = []
    terms = set(title.split())
    for term in terms:
        result.append(vocab_dict[term])
    return result

In [ ]:
idx_list = []
for i, row in df.iterrows():
    idxs = title2idx(row['title'], vocab_dict)
    idx_list.append(idxs)
idx_list

In [ ]:
sim_dict = {}
already = set()
for i in tqdm(range(len(idx_list))):
    if i in already:
        continue
        
    for j in range(i+1,len(idx_list)):
        stand_len = len(idx_list[i]) if len(idx_list[i]) < len(idx_list[j]) else len(idx_list[j])
        inter_len = len(set(idx_list[i]) & set(idx_list[j]))
        if (inter_len / stand_len) >= 0.5:
            if i in sim_dict:
                sim_dict[i].append(j)
            else:
                sim_dict[i] = [j]
            already.add(j)
                
sim_dict

In [ ]:
for stand_idx in sim_dict:
    print(df['title'][stand_idx], stand_idx)
    for idx in sim_dict[stand_idx]:
        print(df['title'][idx])
    print("\n")

In [39]:
multi_document_summary(725)


21 다만 자신의 정리된 마음은 정리되는 대로 여러분께 말씀드리겠다면서, 늘 차분한 마음과 책임있는 마음으로 기다려주기 바란다고 말했습니다.
16 [앵커]이낙연 전 대표 측에선 반발하고 있다고요?[기자]네, 이낙연 캠프 소속 의원 전원은 오늘 경선이 끝난 뒤 긴급회의를 열었습니다.
29 여유롭던 분위기가 달라진 건 30만 명 규모의 3차 선거인단 투표 결과가 발표된 때였습니다.


In [11]:
stand_idx = 586
for idx in sim_dict[stand_idx]:
    print(df['title'][idx])
print("*************")

content_list = []
print(df['content'][stand_idx])
content_list.append(df['content'][stand_idx])
print("************")
for idx in sim_dict[stand_idx]:
    print(df['content'][idx])
    content_list.append(df['content'][idx])
    print("************")

신용카드 캐시백 1천143만명 신청… 9일간 90억원 발생
정부 "신용카드 캐시백 9일간 90억원 발생"
신용카드 캐시백 9일간 90억원 발생…내달 15일 10월분 지급
신용카드 캐시백 9일간 90억원 발생…내달 15일 지급
*************
늘어난 카드 사용액의 10%를 월 최대 10만원까지 돌려주는 신용카드 캐시백(상생소비지원금)이 이달 들어 100억원 가까이 발생한 것으로 나타났다.기획재정부는 다음달 15일 지급될 예정인 10월분 캐시백 지급액이 지난 1~9일 현재 90억원으로 집계됐다고 10일 밝혔다.기재부 관계자는 "참여자 수와 캐시백 발생 구조 등을 고려할 때 캐시백 지급 예정액은 이달 후반으로 갈수록 빠르게 증가할 전망"이라며 "재원 소진 시에는 사업이 조기 종료될 수 있다"고 말했다.캐시백 신청자는 이달 9일 현재 1천143만명으로 집계됐다. 이들 가운데 외국인은 11만명(1.0%)이었다.신청 대비 확정 비율은 99%에 달해 신청자 대부분이 지원 대상 요건을 충족한 것으로 나타났다.카드 캐시백은 월간 카드 사용액이 2분기 월평균 사용액보다 3% 이상 증가하면 초과분의 10%를 캐시백(현금성 충전금)으로 환급해주는 제도다.예를 들어 2분기 월평균 사용액이 100만원이고 10월 카드 사용액이 153만원인 경우, 증가액 53만원 중 3만원(3%)을 제외한 50만원의 10%인 5만원을 캐시백으로 돌려준다.1인당 최대 지급액은 월별 10만원이다.캐시백은 사업 종료 전까지 언제든 신청할 수 있다. 만 19세 이상(2002년 12월 31일 이전 출생자)이고, 2분기 본인 명의 신용·체크카드 사용 실적이 있는 사람이 대상이다.신청 희망자는 롯데·비씨·삼성·신한·우리·KB국민·NH농협 등 9개 카드사 중 하나를 전담 카드사로 지정해 해당 카드사에 신청하면 된다.카드 사용액은 신청 일자와 상관없이 10월 1일 사용분부터 인정되며, 산정 기준이 되는 2분기 카드 사용 실적은 카드사 지정 신청 후 2일 이내에 확인할 수 있다.카드 사용처는 대부분 인정되나 대형

In [12]:
def content_refine_temp(content):
    content = re.sub("\([^\)]*\)","", content)
    content = re.sub("\<[^\]]*\>","", content)
    content = re.sub("[\n\t]+",". ", content)
    content = re.sub("([^\s]*\s기자)","", content)
    content = re.sub("([\w]*@[^\s]*)", "", content)
    content = re.sub("\〈[^〉]*\〉","", content)
    content = re.sub("(\s=\s)","", content)
    content = re.sub("(\s{2,})"," ", content)
    return content

In [13]:
content_refine_temp(content_list[2])

'동영상 뉴스. 정부가 소비 진작책으로 내놓은 소비 신용카드 캐시백, 상생소비지원금이 이달 들어 100억 원 가까이 발생한 것으로 나타났습니다.기획재정부는 다음 달 15일 지급될 예정인 10월분 캐시백 지급액이 이달 들어 9일까지 90억 원으로 집계됐다고 밝혔습니다.카드 캐시백은 월간 카드 사용액이 2분기 월평균 사용액보다 3% 이상 증가하면 초과분의 10%를 캐시백으로 환급해주는 제도입니다.이 제도는 10월과 11월, 두 달 동안 시행되고 1인당 캐시백 한도는 월 10만 원씩 최대 20만 원입니다.연합뉴스TV 기사문의 및 제보 : 카톡/라인 jebo23'

In [14]:
for i in range(len(content_list)):
    content_list[i] = content_refine_temp(content_list[i])
content_list

['늘어난 카드 사용액의 10%를 월 최대 10만원까지 돌려주는 신용카드 캐시백이 이달 들어 100억원 가까이 발생한 것으로 나타났다.기획재정부는 다음달 15일 지급될 예정인 10월분 캐시백 지급액이 지난 1~9일 현재 90억원으로 집계됐다고 10일 밝혔다.기재부 관계자는 "참여자 수와 캐시백 발생 구조 등을 고려할 때 캐시백 지급 예정액은 이달 후반으로 갈수록 빠르게 증가할 전망"이라며 "재원 소진 시에는 사업이 조기 종료될 수 있다"고 말했다.캐시백 신청자는 이달 9일 현재 1천143만명으로 집계됐다. 이들 가운데 외국인은 11만명이었다.신청 대비 확정 비율은 99%에 달해 신청자 대부분이 지원 대상 요건을 충족한 것으로 나타났다.카드 캐시백은 월간 카드 사용액이 2분기 월평균 사용액보다 3% 이상 증가하면 초과분의 10%를 캐시백으로 환급해주는 제도다.예를 들어 2분기 월평균 사용액이 100만원이고 10월 카드 사용액이 153만원인 경우, 증가액 53만원 중 3만원을 제외한 50만원의 10%인 5만원을 캐시백으로 돌려준다.1인당 최대 지급액은 월별 10만원이다.캐시백은 사업 종료 전까지 언제든 신청할 수 있다. 만 19세 이상이고, 2분기 본인 명의 신용·체크카드 사용 실적이 있는 사람이 대상이다.신청 희망자는 롯데·비씨·삼성·신한·우리·KB국민·NH농협 등 9개 카드사 중 하나를 전담 카드사로 지정해 해당 카드사에 신청하면 된다.카드 사용액은 신청 일자와 상관없이 10월 1일 사용분부터 인정되며, 산정 기준이 되는 2분기 카드 사용 실적은 카드사 지정 신청 후 2일 이내에 확인할 수 있다.카드 사용처는 대부분 인정되나 대형마트, 대형 백화점, 대형 종합온라인몰, 대형 전자판매점이나 명품전문매장, 유흥업 사용액 등은 대상에서 배제된다.연회비나 세금, 보험료 등 비소비성 지출도 대상에 포함되지 않는다.',
 '기재부 "1~9일 현재 90억원 집계" 신청자 99% 캐시백 대상 확정늘어난 카드 사용액의 10%를 월 최대 10만원까지 돌려주는 신용카드 캐시백이 

In [15]:
sentence_list = []
for content in content_list:
    for sentence in content.split("다."):
        if sentence == "":
            continue
        else:  
            sentence_list.append(sentence+"다.")
sentence_list

['늘어난 카드 사용액의 10%를 월 최대 10만원까지 돌려주는 신용카드 캐시백이 이달 들어 100억원 가까이 발생한 것으로 나타났다.',
 '기획재정부는 다음달 15일 지급될 예정인 10월분 캐시백 지급액이 지난 1~9일 현재 90억원으로 집계됐다고 10일 밝혔다.',
 '기재부 관계자는 "참여자 수와 캐시백 발생 구조 등을 고려할 때 캐시백 지급 예정액은 이달 후반으로 갈수록 빠르게 증가할 전망"이라며 "재원 소진 시에는 사업이 조기 종료될 수 있다"고 말했다.',
 '캐시백 신청자는 이달 9일 현재 1천143만명으로 집계됐다.',
 ' 이들 가운데 외국인은 11만명이었다.',
 '신청 대비 확정 비율은 99%에 달해 신청자 대부분이 지원 대상 요건을 충족한 것으로 나타났다.',
 '카드 캐시백은 월간 카드 사용액이 2분기 월평균 사용액보다 3% 이상 증가하면 초과분의 10%를 캐시백으로 환급해주는 제도다.',
 '예를 들어 2분기 월평균 사용액이 100만원이고 10월 카드 사용액이 153만원인 경우, 증가액 53만원 중 3만원을 제외한 50만원의 10%인 5만원을 캐시백으로 돌려준다.',
 '1인당 최대 지급액은 월별 10만원이다.',
 '캐시백은 사업 종료 전까지 언제든 신청할 수 있다.',
 ' 만 19세 이상이고, 2분기 본인 명의 신용·체크카드 사용 실적이 있는 사람이 대상이다.',
 '신청 희망자는 롯데·비씨·삼성·신한·우리·KB국민·NH농협 등 9개 카드사 중 하나를 전담 카드사로 지정해 해당 카드사에 신청하면 된다.',
 '카드 사용액은 신청 일자와 상관없이 10월 1일 사용분부터 인정되며, 산정 기준이 되는 2분기 카드 사용 실적은 카드사 지정 신청 후 2일 이내에 확인할 수 있다.',
 '카드 사용처는 대부분 인정되나 대형마트, 대형 백화점, 대형 종합온라인몰, 대형 전자판매점이나 명품전문매장, 유흥업 사용액 등은 대상에서 배제된다.',
 '연회비나 세금, 보험료 등 비소비성 지출도 대상에 포함되지 않는다.',
 '기재부 "1~9일 현재 

In [16]:
vocab_dict = {}
idx = 0
for sentence in sentence_list:
    for term in sentence.split():
        if term not in vocab_dict:
            vocab_dict[term] = idx
            idx+=1
            
terms_list = []
matrix = []
for sentence in sentence_list:
    terms = set(sentence.split())
    vec = [0.0] * len(vocab_dict)
    for term in terms:
        vec[vocab_dict[term]] = 1
    matrix.append(vec)
    terms_list.append(sentence.split())

In [17]:
def cosine_similarity(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(a))

In [18]:
sim_matrix = []
for i in tqdm(range(len(matrix))):
    sim_vec = []
    for j in range(len(matrix)):
        score = cosine_similarity(matrix[i], matrix[j])
        sim_vec.append(score)
    sim_matrix.append(sim_vec)

In [20]:
pd.DataFrame(sim_matrix)

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,1.000000,0.000000,0.058824,0.058824,0.0,0.117647,0.117647,0.117647,0.058824,0.000000,...,0.117647,0.117647,0.058824,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.0
1,0.000000,1.000000,0.066667,0.133333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.037037,0.037037,1.000000,0.074074,0.0,0.000000,0.000000,0.000000,0.000000,0.037037,...,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.037037,0.000000,0.000000,0.0
3,0.142857,0.285714,0.285714,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,0.000000,0.000000,0.000000,0.000000,0.0,0.066667,0.000000,0.066667,0.000000,0.000000,...,0.000000,0.066667,0.000000,0.000000,0.000000,1.000000,0.133333,0.000000,0.066667,0.0
58,0.043478,0.000000,0.043478,0.000000,0.0,0.043478,0.086957,0.130435,0.000000,0.086957,...,0.086957,0.130435,0.000000,0.086957,0.086957,0.086957,1.000000,0.043478,0.000000,0.0
59,0.066667,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,0.066667,0.000000,0.000000,...,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,1.000000,0.000000,0.0
60,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,1.000000,0.0


In [21]:
#representive_scoring
def rep_score_return(sim_vec, idx):
    count = 0
    for i in range(len(sim_vec)):
        if i == idx:
            continue
        else:
            if sim_vec[i] > 0:
                count += 1
    return count / len(sim_vec)

In [22]:
#length_scoring
def len_score_return(terms, max_el, max_rl):
    if max_rl / len(terms) == 1:
        return len(set(terms))/max_el * math.log10(1)
    else:
        return len(set(terms))/max_el * math.log10(max_rl/len(terms))

In [23]:
#diversity_scoring
def div_score_return(sim_vec, idx):
    min_score = 99
    for i, score in enumerate(sim_vec):
        if i!=idx and score > 0 and score < min_score:
            min_score = score
    return min_score

In [24]:
max_el = 0
max_rl = 0
for term in terms_list:
    if len(term) > max_rl:
        max_rl = len(term)
    
    if len(set(term)) > max_el:
        max_el = len(set(term))

score_list = []
for i in range(len(matrix)):
    sim_vec = matrix[i]
    idx = i
    
    rep_score = rep_score_return(sim_vec, idx)
    len_score = len_score_return(terms_list[idx], max_el, max_rl)
    div_score = div_score_return(sim_vec, idx)
    
    score = rep_score * len_score * div_score
#     score = math.log10(rep_score) + math.log10(len_score) + math.log10(div_score)
    score_list.append(score)

In [26]:
temp_idx = np.array(score_list).argsort()[::-1][:2]
temp_idx = sorted(temp_idx)
for sen_idx in temp_idx:
    print(sen_idx,sentence_list[sen_idx])
    print()

30 기획재정부는 다음 달 15일 지급될 예정인 10월분 캐시백 지급액이 이달 들어 9일까지 90억 원으로 집계됐다고 밝혔습니다.

32 이 제도는 10월과 11월, 두 달 동안 시행되고 1인당 캐시백 한도는 월 10만 원씩 최대 20만 원입니다.



In [27]:
def multi_document_summary(stand_idx):
    content_list = []
    content_list.append(df['content'][stand_idx])
    for idx in sim_dict[stand_idx]:
        content_list.append(df['content'][idx])
    for i in range(len(content_list)):
        content_list[i] = content_refine_temp(content_list[i])
    sentence_list = []
    for content in content_list:
        for sentence in content.split("다."):
            if sentence == "":
                continue
            else:  
                sentence_list.append(sentence+"다.")
    vocab_dict = {}
    idx = 0
    for sentence in sentence_list:
        for term in sentence.split():
            if term not in vocab_dict:
                vocab_dict[term] = idx
                idx+=1

    terms_list = []
    matrix = []
    for sentence in sentence_list:
        terms = set(sentence.split())
        vec = [0.0] * len(vocab_dict)
        for term in terms:
            vec[vocab_dict[term]] = 1
        matrix.append(vec)
        terms_list.append(sentence.split())
    sim_matrix = []
    for i in tqdm(range(len(matrix))):
        sim_vec = []
        for j in range(len(matrix)):
            score = cosine_similarity(matrix[i], matrix[j])
            sim_vec.append(score)
        sim_matrix.append(sim_vec)
    max_el = 0
    max_rl = 0
    for term in terms_list:
        if len(term) > max_rl:
            max_rl = len(term)

        if len(set(term)) > max_el:
            max_el = len(set(term))

    score_list = []
    for i in range(len(matrix)):
        sim_vec = matrix[i]
        idx = i

        rep_score = rep_score_return(sim_vec, idx)
        len_score = len_score_return(terms_list[idx], max_el, max_rl)
        div_score = div_score_return(sim_vec, idx)

        score = rep_score * len_score * div_score
        score_list.append(score)
    temp_idx = np.array(score_list).argsort()[::-1][:3]
#     temp_idx = sorted(temp_idx)
    for sen_idx in temp_idx:
        print(sen_idx,sentence_list[sen_idx])